<a href="https://colab.research.google.com/github/hsw1805/ML-DL/blob/main/05_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#교차 검증

## 검증 세트

데이터가 100개가 있다고 가정하자

80:20 비율로 80은 train set, 20은 test set으로 구분하자

또 train set으로 구분한 80개 중 60:20 비율로 train set과 validation set으로 구분한다

train set = 기출문제

validation set = 모의고사

test set = 수능

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target=train_test_split(data, target,test_size=0.2, random_state =42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state= 42)

## DecisionTreeClassifier로 분류

In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [6]:
from sklearn.model_selection import cross_validate

scores =  cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01079416, 0.01030469, 0.01074505, 0.01062369, 0.01018286]), 'score_time': array([0.0015254 , 0.00143147, 0.0013833 , 0.0016427 , 0.00157905]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
# 섞어서 데이터 교차 검증
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
#10개의 Fold로 분류
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state =42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼 파라미터 튜닝

개별 하이퍼 파라미터는 다른 하이퍼 파라미터의 값에 영향을 받기 때문에 다 같이 찾아야함

GridSearchCV를 통해 하이퍼파라미터 값을 찾게 도와줌

In [10]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
#CrossValidation (CV)의 매개변수는 기본으로 5가 들어가있음
#n_jobs 은 몇개의 cpu를 사용할 것인가
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)

gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [11]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [12]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][0])

{'min_impurity_decrease': 0.0001}


In [14]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split': range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params,n_jobs = -1)

gs.fit(train_input, train_target)

print(gs.best_params_)



{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [15]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [16]:
print(gs.cv_results_)

{'mean_fit_time': array([0.00864568, 0.00964775, 0.01197138, ..., 0.00930204, 0.00958543,
       0.00722461]), 'std_fit_time': array([0.00408018, 0.00251246, 0.00280671, ..., 0.00342057, 0.00238793,
       0.00101787]), 'mean_score_time': array([0.00239849, 0.00414224, 0.00147376, ..., 0.00179677, 0.00304914,
       0.0024756 ]), 'std_score_time': array([1.03364161e-03, 2.26690466e-03, 5.96131509e-05, ...,
       3.90350309e-04, 3.31178834e-03, 1.83899025e-03]), 'param_max_depth': masked_array(data=[5, 5, 5, ..., 19, 19, 19],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0001, 0.0001, ..., 0.0009000000000000001,
                   0.0009000000000000001, 0.0009000000000000001],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_split': masked_array(data=[2, 12, 22, ..., 72, 

In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


##랜덤 서치

In [22]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
print(rgen)
rgen.rvs(20)

array([9, 6, 8, 9, 2, 4, 6, 1, 7, 8, 4, 0, 4, 4, 2, 4, 1, 0, 3, 7])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [26]:
ugen = uniform(0,1)
ugen.rvs(10)

array([5.71419417, 2.33899968, 8.33140334, 7.15221239, 5.49940872,
       4.02843415, 6.50055235, 6.21037989, 7.98897469, 7.18569576])

In [33]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf' : randint(1,25),
          }

In [34]:
from sklearn.model_selection import RandomizedSearchCV
#위의 param으로 100번의 난수를 돌려 가장 최적의 param의 값을 찾기
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter = 100, n_jobs = -1, random_state = 42)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a33f614acb0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a33f6149ba0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a33f6149510>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a33f614a3e0>},
                   random_state=42)

In [36]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [37]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [38]:
#가장 좋은 성능을 내는 params값으로 만든 DT
#이걸로 testscore 평가
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
